In [1]:
!pip install eli5

In [2]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df=pd.read_csv('data/men_shoes.csv',low_memory=False)

In [5]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats,model=DecisionTreeRegressor(max_depth=5)):
  X=df[feats].values
  y=df.prices_amountmin.values
  scores=cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [0]:
df['brand_cat']=df.brand.map(lambda x:str(x).lower()).factorize()[0]

In [8]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model=RandomForestRegressor(n_estimators=100,max_depth=5,random_state=0)

In [10]:
run_model(['brand_cat'],model=model)

(-57.31783843165656, 4.181246596160967)

In [11]:
literal_eval(df.features.head().values[0])[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
   if str(x)=='nan':return list()
   return literal_eval(x.replace('\\\\"',''))

df['features_parsed']=df.features.map(parse_features)

In [13]:
def parse_2(x):
  return {list(y.values())[0].lower():list(y.values())[1][0].lower() for y in x}

df.features_parsed=df.features_parsed.map(parse_2)
df.features_parsed

0        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2        {'gender': 'men', 'color': 'black', 'shipping ...
3        {'gender': 'men', 'color': 'black', 'shipping ...
4        {'gender': 'men', 'color': 'black', 'shipping ...
                               ...                        
18275    {'style': 'desert boots', 'pattern': 'solid', ...
18276    {'shoe size': '9.5', 'gender': 'men', 'shoe ca...
18277    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18278    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18279    {'gender': 'unisex', 'frame style': 'wrap', 'p...
Name: features_parsed, Length: 18280, dtype: object

In [14]:
keys=set()
df.features_parsed.map(lambda x: keys.update(x.keys()))
len(keys)

476

In [0]:
def get_name_feat(key):
  return 'feat_'+key

In [16]:
for key in tqdm_notebook(keys):
  df[get_name_feat(key)]=df.features_parsed.map(lambda feats:feats[key] if key in feats else np.nan)

In [17]:
df[False==df.feat_athlete.isnull()].shape[0]

8

In [0]:
key_stat={}
for key in keys:
  key_stat[key]=df[False==df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [19]:
{k:v for k,v in key_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_color_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]
df['feat_sport_cat']=df['feat_sport'].factorize()[0]
df['feat_style_cat']=df['feat_style'].factorize()[0]

In [0]:
df.brand=df.brand.map(lambda x:str(x).lower())

In [22]:
model=RandomForestRegressor(max_depth=5,n_estimators=100)
run_model(['brand_cat'],model)

(-57.30026450626942, 4.197711410484459)

In [0]:
model=RandomForestRegressor(max_depth=5,n_estimators=100)
result=run_model(['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_manufacturer part number_cat','feat_material_cat'],model)

In [0]:
feats=['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_manufacturer part number_cat','feat_material_cat','feat_sport_cat','feat_style_cat']

In [35]:
feats_cat=[x for x in df.columns if "cat" in x]
feats_cat

['categories',
 'brand_cat',
 'feat_location - city/state',
 'feat_recommended location',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_shoe category',
 'feat_fabrication',
 'feat_catalog',
 'feat_certifications and listings',
 'feat_location - country',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_dial color_cat',
 'feat_finish_cat',
 'feat_impact resistant_cat',
 'feat_gold karat_cat',
 'feat_box_cat',
 'feat_contained battery type_cat',
 'feat_gemstone type_cat',
 'feat_is polarized_cat',
 'feat_picture_cat',
 'feat_band manufacturer_cat',
 'feat_footwear type_cat',
 'feat_manufacturer number_cat',
 'feat_resizable_cat',
 'feat_insulated_cat',
 'feat_fits models_cat',
 'feat_hairstyle_cat',
 'feat_neckline_cat',
 'feat_expandable_cat',
 'feat_name_cat',
 'feat_eyewear frame style_cat',
 'feat_product dimensions_cat',
 'feat_is dark sky-compliant_cat

In [39]:
X=df[feats].values
y=df.prices_amountmin.values
m=RandomForestRegressor(n_estimators=100,max_depth=5,random_state=0)
m.fit(X,y)
print(result)
perm=PermutationImportance(m,random_state=1).fit(X,y)
eli5.show_weights(perm,feature_names=feats)

(-57.069647213957616, 4.318529075926194)


Weight,Feature
0.2524 ± 0.0075,brand_cat
0.1018 ± 0.0109,feat_material_cat
0.0451 ± 0.0015,feat_gender_cat
0.0200 ± 0.0007,feat_brand_cat
0.0070 ± 0.0012,feat_color_cat
0.0068 ± 0.0010,feat_style_cat
0.0006 ± 0.0001,feat_manufacturer part number_cat
0.0003 ± 0.0000,feat_sport_cat


In [26]:
df.brand.value_counts(normalize=True)

nike               0.097210
puma               0.033315
ralph lauren       0.028775
vans               0.021116
new balance        0.020295
                     ...   
mod activewear     0.000055
hanro              0.000055
tallia             0.000055
old west           0.000055
static footwear    0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
for key in keys:
  df[get_name_feat(key)+'_cat']=df[get_name_feat(key)].factorize()[0]

In [40]:
!git add day5.ipynb

'/content/drive/My Drive/Colab Notebooks/dw_matrix'